# Step 2: Tool 정의하기

이 노트북에서는 AI가 호출할 수 있는 **도구(Tool)**를 정의하는 방법을 배웁니다.

**학습 목표:**
- Tool이란 무엇인가?
- Tool Schema 작성법
- AI에게 Tool 등록하기

---

## 1. Tool이란?

**Tool**은 AI가 호출할 수 있는 외부 함수입니다.

```
일반 AI:    "현재 시간은 제가 알 수 없습니다."
Tool 사용:  [get_time 호출] → "현재 시간은 14:30입니다."
```

AI는 학습 데이터에 없는 실시간 정보(시간, 날씨, 계산 등)를 Tool을 통해 얻을 수 있습니다.

---

## 2. 패키지 설치 및 모델 선택

In [1]:
!pip install anthropic -q
print("설치 완료!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 5.7 MB/s eta 0:00:00
설치 완료!


In [2]:
# ===== Claude 4.5 모델 선택 =====
# 아래 세 가지 모델 중 하나를 선택하세요 (주석 해제)

# 1. Haiku 4.5 - 가장 빠르고 저렴 (테스트용 추천)
# MODEL = "claude-haiku-4-5-20251001"

# 2. Sonnet 4.5 - 균형 잡힌 성능
MODEL = "claude-sonnet-4-5-20250929"

# 3. Opus 4.5 - 최고 성능 (비용 높음)
# MODEL = "claude-opus-4-5-20251101"

print(f"선택된 모델: {MODEL}")

선택된 모델: claude-sonnet-4-5-20250929


---

## 3. API 키 설정

In [3]:
import os
from anthropic import Anthropic

# API 키 설정 (둘 중 하나 선택)
# 방법 1: 직접 입력
# os.environ["ANTHROPIC_API_KEY"] = "sk-ant-여기에-API-키-입력"

# 방법 2: Colab Secrets
try:
    from google.colab import userdata
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
except:
    pass

client = Anthropic()
print("준비 완료!")

준비 완료!


---

## 4. 첫 번째 Tool 정의

Tool은 **JSON Schema** 형식으로 정의합니다.

### 필수 요소
- `name`: 도구 이름 (영어, snake_case)
- `description`: 도구 설명 (AI가 읽고 이해함)
- `input_schema`: 입력 파라미터 정의

In [4]:
# 가장 간단한 Tool: 현재 시간 조회 (파라미터 없음)
GET_TIME_TOOL = {
    "name": "get_current_time", # 실행할 함수의 이름으로
    "description": "현재 날짜와 시간을 조회합니다.", # llm이 어떤 함수를 판단해야 할지 참고하는 지침
    "input_schema": {
        "type": "object", # 반드시 object
        "properties": {}  # 파라미터 없음
    }
}

print("Tool 정의 완료!")
print(f"  이름: {GET_TIME_TOOL['name']}")
print(f"  설명: {GET_TIME_TOOL['description']}")

Tool 정의 완료!
  이름: get_current_time
  설명: 현재 날짜와 시간을 조회합니다.


---

## 5. 파라미터가 있는 Tool

대부분의 Tool은 입력 파라미터가 필요합니다.

In [5]:
# 파라미터가 있는 Tool: 날씨 조회
GET_WEATHER_TOOL = {
    "name": "get_weather",
    "description": "특정 도시의 현재 날씨를 조회합니다.",
    "input_schema": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "날씨를 조회할 도시 이름 (예: 서울, 부산)"
            }
        },
        "required": ["city"]  # 필수 파라미터
    }
}

print("날씨 Tool 정의 완료!")
print(f"  필수 파라미터: {GET_WEATHER_TOOL['input_schema']['required']}")

날씨 Tool 정의 완료!
  필수 파라미터: ['city']


---

## 6. 계산기 Tool (여러 파라미터)

`enum`을 사용하면 선택지를 제한할 수 있습니다.

In [6]:
# 여러 파라미터 + enum 사용
CALCULATOR_TOOL = {
    "name": "calculator",
    "description": "두 숫자의 사칙연산을 수행합니다.",
    "input_schema": {
        "type": "object",
        "properties": {
            "a": {
                "type": "number",
                "description": "첫 번째 숫자"
            },
            "b": {
                "type": "number",
                "description": "두 번째 숫자"
            },
            "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],  # 선택지 제한
                "description": "수행할 연산"
            }
        },
        "required": ["a", "b", "operation"]
    }
}

print("계산기 Tool 정의 완료!")
print(f"  연산 종류: {CALCULATOR_TOOL['input_schema']['properties']['operation']['enum']}")

계산기 Tool 정의 완료!
  연산 종류: ['add', 'subtract', 'multiply', 'divide']


---

## 7. AI에게 Tool 등록하고 호출 확인

Tool을 API 호출 시 `tools` 파라미터로 전달하면, AI가 필요할 때 호출합니다.

In [7]:
# 서울 날씨 알려줘 -> tools 안에 있는 함수 목록을 봅니다, 특히 거기서 description을 봅니다.
# 만약에 해당 프롬프트가 tools의 description으로 설명이 가능하다면, ai는 답변을 구구절절 생성하는 게 아니라
# 이 툴 쓰세요, 라고 말하고 끝입니다.
# Tool을 등록하여 API 호출
messages = [
        {"role": "user", "content": "서울 날씨 알려줘"}
    ]
response = client.messages.create(
    model=MODEL,
    max_tokens=200,
    tools=[GET_WEATHER_TOOL],  # Tool 등록!
    messages=messages
)

print(f"stop_reason: {response.stop_reason}")
print()

stop_reason: tool_use



In [8]:
response.content[0]

ToolUseBlock(id='toolu_01B9sVBiS3QEBmCGZdVKf8gQ', input={'city': '서울'}, name='get_weather', type='tool_use')

### ToolUseBlock 분석

위 결과에서 `ToolUseBlock`의 구조를 확인할 수 있습니다:

| 속성 | 설명 | 예시 값 |
|:-----|:-----|:--------|
| `id` | 도구 호출의 고유 ID | `toolu_01B9sVBiS3QE...` |
| `name` | 호출할 도구 이름 | `get_weather` |
| `input` | 도구에 전달할 파라미터 | `{'city': '서울'}` |
| `type` | 블록 타입 (항상 `tool_use`) | `tool_use` |

이제 `response.content`를 순회하면서 모든 블록을 확인해봅시다.

In [9]:
# AI가 Tool을 호출하려고 함!
for block in response.content:
    print(f"블록 타입: {block.type}")

    if block.type == "tool_use":
        print(f"  호출할 도구: {block.name}")
        print(f"  파라미터: {block.input}")

블록 타입: tool_use
  호출할 도구: get_weather
  파라미터: {'city': '서울'}


### 핵심 포인트

- `stop_reason`이 `"tool_use"`이면 AI가 도구를 호출하려는 것!
- `block.type`이 `"tool_use"`인 블록에서 도구 정보 확인
- `block.name`: 호출할 도구 이름
- `block.input`: 도구에 전달할 파라미터

---

## 8. description의 중요성

AI는 `description`을 읽고 어떤 도구를 사용할지 결정합니다.

In [10]:
# 나쁜 예 vs 좋은 예

BAD = "날씨"  # 너무 짧음

GOOD = """특정 도시의 현재 날씨(온도, 습도, 상태)를 조회합니다.
예시: "서울 날씨" → city="서울"
예시: "부산 날씨 알려줘" → city="부산"
"""

print("나쁜 description:")
print(f"  '{BAD}'")
print()
print("좋은 description:")
print(f"  '{GOOD}'")

나쁜 description:
  '날씨'

좋은 description:
  '특정 도시의 현재 날씨(온도, 습도, 상태)를 조회합니다.
예시: "서울 날씨" → city="서울"
예시: "부산 날씨 알려줘" → city="부산"
'


In [11]:
# ===== 셀 3: 나쁜 Description vs 좋은 Description 정의 =====

# 나쁜 Description
WEATHER_TOOL_BAD = {
    "name": "get_weather",
    "description": "날씨",  # 너무 짧고 모호함
    "input_schema": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "도시"  # 역시 모호함
            }
        },
        "required": ["city"]
    }
}

# 좋은 Description
WEATHER_TOOL_GOOD = {
    "name": "get_weather",
    "description": """특정 도시의 현재 날씨 정보(온도, 습도, 날씨 상태)를 조회합니다.
사용자가 날씨, 기온, 온도, 비, 눈 등을 물어볼 때 이 도구를 사용하세요.
예시: "서울 날씨 어때?" → city="서울"
예시: "부산 오늘 비 와?" → city="부산" """,
    "input_schema": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "날씨를 조회할 도시 이름 (예: 서울, 부산, 대구, 인천)"
            }
        },
        "required": ["city"]
    }
}

print("Tool 정의 완료!")
print("\n[나쁜 Description]")
print(f"  Tool: {WEATHER_TOOL_BAD['description']}")
print(f"  Param: {WEATHER_TOOL_BAD['input_schema']['properties']['city']['description']}")
print("\n[좋은 Description]")
print(f"  Tool: {WEATHER_TOOL_GOOD['description'][:50]}...")
print(f"  Param: {WEATHER_TOOL_GOOD['input_schema']['properties']['city']['description']}")

Tool 정의 완료!

[나쁜 Description]
  Tool: 날씨
  Param: 도시

[좋은 Description]
  Tool: 특정 도시의 현재 날씨 정보(온도, 습도, 날씨 상태)를 조회합니다.
사용자가 날씨, 기온...
  Param: 날씨를 조회할 도시 이름 (예: 서울, 부산, 대구, 인천)


In [12]:
def test_tool_calling(tool, user_message):
    """Tool 호출 테스트 함수"""
    response = client.messages.create(
        model=MODEL,
        max_tokens=300,
        tools=[tool],
        messages=[{"role": "user", "content": user_message}]
    )

    result = {
        "stop_reason": response.stop_reason,
        "tool_called": False,
        "tool_name": None,
        "tool_input": None,
        "text_response": None
    }

    for block in response.content:
        if block.type == "tool_use":
            result["tool_called"] = True
            result["tool_name"] = block.name
            result["tool_input"] = block.input
        elif block.type == "text":
            result["text_response"] = block.text[:100] if block.text else None

    return result



In [13]:
# ===== 셀 5: 실험 1 - 나쁜 Description 테스트 =====

print("="*60)
print("[실험 1] 나쁜 Description 테스트")
print("="*60)

test_messages = [
    "서울 날씨 알려줘",
    "오늘 부산 비 올까?",
    "제주도 기온이 어떻게 돼?"
]

for msg in test_messages:
    result = test_tool_calling(WEATHER_TOOL_BAD, msg)
    print(f"\n입력: \"{msg}\"")
    print(f"  Tool 호출: {result['tool_called']}")
    if result['tool_called']:
        print(f"  파라미터: {result['tool_input']}")
    else:
        print(f"  텍스트 응답: {result['text_response']}")

[실험 1] 나쁜 Description 테스트

입력: "서울 날씨 알려줘"
  Tool 호출: True
  파라미터: {'city': '서울'}

입력: "오늘 부산 비 올까?"
  Tool 호출: True
  파라미터: {'city': '부산'}

입력: "제주도 기온이 어떻게 돼?"
  Tool 호출: True
  파라미터: {'city': '제주도'}


In [14]:
# ===== 셀 6: 실험 2 - 좋은 Description 테스트 =====

print("="*60)
print("[실험 2] 좋은 Description 테스트")
print("="*60)

for msg in test_messages:
    result = test_tool_calling(WEATHER_TOOL_GOOD, msg)
    print(f"\n입력: \"{msg}\"")
    print(f"  Tool 호출: {result['tool_called']}")
    if result['tool_called']:
        print(f"  파라미터: {result['tool_input']}")
    else:
        print(f"  텍스트 응답: {result['text_response']}")

[실험 2] 좋은 Description 테스트

입력: "서울 날씨 알려줘"
  Tool 호출: True
  파라미터: {'city': '서울'}

입력: "오늘 부산 비 올까?"
  Tool 호출: True
  파라미터: {'city': '부산'}

입력: "제주도 기온이 어떻게 돼?"
  Tool 호출: True
  파라미터: {'city': '제주'}


In [15]:
# ===== 셀 7: Enum 없는 Calculator vs Enum 있는 Calculator =====

# Enum 없는 Calculator
CALC_WITHOUT_ENUM = {
    "name": "calculator",
    "description": "두 숫자의 사칙연산을 수행합니다.",
    "input_schema": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "첫 번째 숫자"},
            "b": {"type": "number", "description": "두 번째 숫자"},
            "operation": {
                "type": "string",
                "description": "수행할 연산"  # enum 없음!
            }
        },
        "required": ["a", "b", "operation"]
    }
}

# Enum 있는 Calculator
CALC_WITH_ENUM = {
    "name": "calculator",
    "description": "두 숫자의 사칙연산을 수행합니다.",
    "input_schema": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "첫 번째 숫자"},
            "b": {"type": "number", "description": "두 번째 숫자"},
            "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],  # enum 있음!
                "description": "수행할 연산 (add, subtract, multiply, divide 중 선택)"
            }
        },
        "required": ["a", "b", "operation"]
    }
}

print("Calculator Tool 정의 완료!")
print(f"\n[Enum 없음] operation 타입: string (자유 입력)")
print(f"[Enum 있음] operation 선택지: {CALC_WITH_ENUM['input_schema']['properties']['operation']['enum']}")

Calculator Tool 정의 완료!

[Enum 없음] operation 타입: string (자유 입력)
[Enum 있음] operation 선택지: ['add', 'subtract', 'multiply', 'divide']


In [29]:
# ===== 셀 8: 실험 3 - Enum 없을 때 테스트 =====

print("="*60)
print("[실험 3] Enum 없는 Calculator 테스트")
print("="*60)

calc_messages = [
    "10 더하기 5 계산해줘",
    "100에서 30 빼줘",
    "7 곱하기 8",
    "50 나누기 2"
]

print("\n⚠️  Enum이 없으면 Claude가 다양한 형태로 operation을 생성할 수 있습니다:")
print("   예: 'add', 'plus', '더하기', 'addition', '+' 등\n")

for msg in calc_messages:
    result = test_tool_calling(CALC_WITHOUT_ENUM, msg)
    print(f"입력: \"{msg}\"")
    if result['tool_called']:
        print(f"  → operation: \"{result['tool_input'].get('operation')}\"")
    print()

[실험 3] Enum 없는 Calculator 테스트

⚠️  Enum이 없으면 Claude가 다양한 형태로 operation을 생성할 수 있습니다:
   예: 'add', 'plus', '더하기', 'addition', '+' 등

입력: "10 더하기 5 계산해줘"
  → operation: "add"

입력: "100에서 30 빼줘"
  → operation: "subtract"

입력: "7 곱하기 8"
  → operation: "곱하기"

입력: "50 나누기 2"
  → operation: "divide"



In [17]:
# ===== 셀 9: 실험 4 - Enum 있을 때 테스트 =====

print("="*60)
print("[실험 4] Enum 있는 Calculator 테스트")
print("="*60)

print("\n✅ Enum이 있으면 반드시 정해진 값 중 하나만 선택됩니다:")
print(f"   허용 값: {CALC_WITH_ENUM['input_schema']['properties']['operation']['enum']}\n")

for msg in calc_messages:
    result = test_tool_calling(CALC_WITH_ENUM, msg)
    print(f"입력: \"{msg}\"")
    if result['tool_called']:
        op = result['tool_input'].get('operation')
        is_valid = op in ["add", "subtract", "multiply", "divide"]
        status = "✅" if is_valid else "❌"
        print(f"  → operation: \"{op}\" {status}")
    print()

[실험 4] Enum 있는 Calculator 테스트

✅ Enum이 있으면 반드시 정해진 값 중 하나만 선택됩니다:
   허용 값: ['add', 'subtract', 'multiply', 'divide']

입력: "10 더하기 5 계산해줘"
  → operation: "add" ✅

입력: "100에서 30 빼줘"
  → operation: "subtract" ✅

입력: "7 곱하기 8"
  → operation: "multiply" ✅

입력: "50 나누기 2"
  → operation: "divide" ✅



In [18]:
# ===== 셀 10: 세 가지 Tool 정의 (완전한 예제용) =====

TOOLS = [
    # Tool 1: 현재 시간
    {
        "name": "get_current_time",
        "description": """현재 날짜와 시간을 조회합니다.
사용자가 '지금 몇 시야?', '오늘 날짜', '현재 시간' 등을 물어볼 때 사용합니다.""",
        "input_schema": {
            "type": "object",
            "properties": {}  # 파라미터 없음
        }
    },

    # Tool 2: 날씨
    {
        "name": "get_weather",
        "description": """특정 도시의 현재 날씨 정보를 조회합니다.
사용자가 날씨, 기온, 비, 눈 등을 물어볼 때 사용합니다.
예시: "서울 날씨" → city="서울" """,
        "input_schema": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "날씨를 조회할 도시 이름"
                }
            },
            "required": ["city"]
        }
    },

    # Tool 3: 계산기
    {
        "name": "calculator",
        "description": """두 숫자의 사칙연산을 수행합니다.
사용자가 계산, 더하기, 빼기, 곱하기, 나누기를 요청할 때 사용합니다.""",
        "input_schema": {
            "type": "object",
            "properties": {
                "a": {"type": "number", "description": "첫 번째 숫자"},
                "b": {"type": "number", "description": "두 번째 숫자"},
                "operation": {
                    "type": "string",
                    "enum": ["add", "subtract", "multiply", "divide"],
                    "description": "수행할 연산"
                }
            },
            "required": ["a", "b", "operation"]
        }
    }
]

print("✅ 3개의 Tool 정의 완료!")
for tool in TOOLS:
    print(f"  - {tool['name']}")

✅ 3개의 Tool 정의 완료!
  - get_current_time
  - get_weather
  - calculator


In [19]:
# ===== 셀 11: Mock 함수 구현 =====

from datetime import datetime
import random

def get_current_time() -> str:
    """현재 시간 반환 (실제 시간 사용)"""
    now = datetime.now()
    return f"현재 시간: {now.strftime('%Y년 %m월 %d일 %H시 %M분 %S초')}"


def get_weather(city: str) -> str:
    """날씨 반환 (Mock 데이터)"""
    weather_data = {
        "서울": {"temp": 22, "status": "맑음", "humidity": 45},
        "부산": {"temp": 25, "status": "구름 조금", "humidity": 60},
        "제주": {"temp": 27, "status": "흐림", "humidity": 75},
        "대구": {"temp": 28, "status": "맑음", "humidity": 40},
        "인천": {"temp": 21, "status": "비", "humidity": 80},
    }

    if city in weather_data:
        data = weather_data[city]
        return f"{city} 날씨: {data['status']}, 기온 {data['temp']}°C, 습도 {data['humidity']}%"
    else:
        temp = random.randint(15, 30)
        status = random.choice(["맑음", "흐림", "비", "눈"])
        return f"{city} 날씨: {status}, 기온 {temp}°C"


def calculator(a: float, b: float, operation: str) -> str:
    """계산 수행"""
    if operation == "add":
        result = a + b
        symbol = "+"
    elif operation == "subtract":
        result = a - b
        symbol = "-"
    elif operation == "multiply":
        result = a * b
        symbol = "×"
    elif operation == "divide":
        if b == 0:
            return "오류: 0으로 나눌 수 없습니다."
        result = a / b
        symbol = "÷"
    else:
        return f"오류: 알 수 없는 연산 '{operation}'"

    return f"{a} {symbol} {b} = {result}"


print("✅ Mock 함수 3개 정의 완료!")
print("\n테스트:")
print(f"  {get_current_time()}")
print(f"  {get_weather('서울')}")
print(f"  {calculator(10, 5, 'add')}")

✅ Mock 함수 3개 정의 완료!

테스트:
  현재 시간: 2025년 12월 04일 07시 28분 05초
  서울 날씨: 맑음, 기온 22°C, 습도 45%
  10 + 5 = 15


In [20]:
# ===== 셀 12: Tool 실행 함수 =====

def execute_tool(tool_name: str, tool_input: dict) -> str:
    """Tool 이름에 따라 적절한 함수를 실행하고 결과 반환"""

    if tool_name == "get_current_time":
        return get_current_time()

    elif tool_name == "get_weather":
        city = tool_input.get("city", "서울")
        return get_weather(city)

    elif tool_name == "calculator":
        a = tool_input.get("a", 0)
        b = tool_input.get("b", 0)
        operation = tool_input.get("operation", "add")
        return calculator(a, b, operation)

    else:
        return f"오류: 알 수 없는 도구 '{tool_name}'"


print("✅ Tool 실행 함수 정의 완료!")

✅ Tool 실행 함수 정의 완료!


In [21]:
# ===== 셀 13: 완전한 Agentic Loop 구현 =====

def run_agent(user_message: str, verbose: bool = True) -> str:
    """
    완전한 Agentic Loop:
    1. 사용자 메시지를 Claude에게 전달
    2. Claude가 Tool 호출을 요청하면 실행
    3. 실행 결과를 Claude에게 전달
    4. 최종 응답 반환
    """

    if verbose:
        print(f"\n{'='*60}")
        print(f"👤 사용자: {user_message}")
        print(f"{'='*60}")

    messages = [{"role": "user", "content": user_message}]

    # Step 1: 첫 번째 API 호출
    response = client.messages.create(
        model=MODEL,
        max_tokens=500,
        tools=TOOLS,
        messages=messages
    )

    if verbose:
        print(f"\n📡 Claude 응답 (stop_reason: {response.stop_reason})")

    # Step 2: Tool 호출이 필요한 경우
    if response.stop_reason == "tool_use":
        # Tool 호출 정보 추출
        tool_use_block = None

        for block in response.content:
            print(block)
            if block.type == "tool_use":
                tool_use_block = block
                break

        if tool_use_block:
            tool_name = tool_use_block.name
            tool_input = tool_use_block.input
            tool_use_id = tool_use_block.id

            if verbose:
                print(f"\n🔧 Tool 호출 요청:")
                print(f"   도구: {tool_name}")
                print(f"   입력: {tool_input}")

            # Step 3: Tool 실행
            tool_result = execute_tool(tool_name, tool_input)

            if verbose:
                print(f"\n⚡ Tool 실행 결과:")
                print(f"   {tool_result}")

            # Step 4: 결과를 Claude에게 전달
            messages.append({"role": "assistant", "content": response.content})
            messages.append({
                "role": "user",
                "content": [{
                    "type": "tool_result",
                    "tool_use_id": tool_use_id,
                    "content": tool_result
                }]
            })

            # Step 5: 최종 응답 받기
            final_response = client.messages.create(
                model=MODEL,
                max_tokens=500,
                tools=TOOLS,
                messages=messages
            )

            # 최종 텍스트 추출
            final_text = ""
            for block in final_response.content:
                if block.type == "text":
                    final_text = block.text
                    break

            if verbose:
                print(f"\n🤖 Claude 최종 응답:")
                print(f"   {final_text}")

            return final_text

    # Tool 호출 없이 바로 응답한 경우
    else:
        for block in response.content:
            if block.type == "text":
                if verbose:
                    print(f"\n🤖 Claude 응답 (Tool 미사용):")
                    print(f"   {block.text}")
                return block.text

    return "응답을 받지 못했습니다."


print("✅ Agentic Loop 함수 정의 완료!")

✅ Agentic Loop 함수 정의 완료!


In [22]:
# ===== 셀 14: 테스트 - 시간 질문 =====
run_agent("지금 몇 시야?")


👤 사용자: 지금 몇 시야?

📡 Claude 응답 (stop_reason: tool_use)
ToolUseBlock(id='toolu_01CtYcWJ8Yq8GMaAU1zHDX3b', input={}, name='get_current_time', type='tool_use')

🔧 Tool 호출 요청:
   도구: get_current_time
   입력: {}

⚡ Tool 실행 결과:
   현재 시간: 2025년 12월 04일 07시 28분 08초

🤖 Claude 최종 응답:
   지금은 **2025년 12월 04일 07시 28분**입니다.


'지금은 **2025년 12월 04일 07시 28분**입니다.'

In [23]:
# ===== 셀 15: 테스트 - 날씨 질문 =====
run_agent("서울 날씨 어때?")


👤 사용자: 서울 날씨 어때?

📡 Claude 응답 (stop_reason: tool_use)
ToolUseBlock(id='toolu_01XwhTzWKRNL968d2j4HLKQn', input={'city': '서울'}, name='get_weather', type='tool_use')

🔧 Tool 호출 요청:
   도구: get_weather
   입력: {'city': '서울'}

⚡ Tool 실행 결과:
   서울 날씨: 맑음, 기온 22°C, 습도 45%

🤖 Claude 최종 응답:
   서울의 현재 날씨는 **맑음**이에요! ☀️

- **기온**: 22°C
- **습도**: 45%

날씨가 화창하고 쾌적하네요. 외출하기 좋은 날씨입니다!


'서울의 현재 날씨는 **맑음**이에요! ☀️\n\n- **기온**: 22°C\n- **습도**: 45%\n\n날씨가 화창하고 쾌적하네요. 외출하기 좋은 날씨입니다!'

In [24]:
# ===== 셀 16: 테스트 - 다른 도시 날씨 =====
run_agent("부산 오늘 비 올까?")


👤 사용자: 부산 오늘 비 올까?

📡 Claude 응답 (stop_reason: tool_use)
ToolUseBlock(id='toolu_011ixJ9kXD4MYDr4jZz5F1eV', input={'city': '부산'}, name='get_weather', type='tool_use')

🔧 Tool 호출 요청:
   도구: get_weather
   입력: {'city': '부산'}

⚡ Tool 실행 결과:
   부산 날씨: 구름 조금, 기온 25°C, 습도 60%

🤖 Claude 최종 응답:
   부산 오늘 날씨는 구름이 조금 있고, 기온은 25°C, 습도는 60%입니다. 비는 오지 않을 것 같네요! ☀️


'부산 오늘 날씨는 구름이 조금 있고, 기온은 25°C, 습도는 60%입니다. 비는 오지 않을 것 같네요! ☀️'

In [25]:
# ===== 셀 17: 테스트 - 계산 (덧셈) =====
run_agent("123 더하기 456 계산해줘")


👤 사용자: 123 더하기 456 계산해줘

📡 Claude 응답 (stop_reason: tool_use)
ToolUseBlock(id='toolu_01T99MyhZq3TaF5Vdreikb9U', input={'a': 123, 'b': 456, 'operation': 'add'}, name='calculator', type='tool_use')

🔧 Tool 호출 요청:
   도구: calculator
   입력: {'a': 123, 'b': 456, 'operation': 'add'}

⚡ Tool 실행 결과:
   123 + 456 = 579

🤖 Claude 최종 응답:
   123 더하기 456은 **579**입니다.


'123 더하기 456은 **579**입니다.'

In [26]:
# ===== 셀 19: 테스트 - 계산 (나눗셈) =====
run_agent("100을 8로 나눠줘")


👤 사용자: 100을 8로 나눠줘

📡 Claude 응답 (stop_reason: tool_use)
ToolUseBlock(id='toolu_01BaR68NF9nmzCKDgTpiHhEf', input={'a': 100, 'b': 8, 'operation': 'divide'}, name='calculator', type='tool_use')

🔧 Tool 호출 요청:
   도구: calculator
   입력: {'a': 100, 'b': 8, 'operation': 'divide'}

⚡ Tool 실행 결과:
   100 ÷ 8 = 12.5

🤖 Claude 최종 응답:
   100을 8로 나눈 결과는 **12.5**입니다.


'100을 8로 나눈 결과는 **12.5**입니다.'

In [27]:
# ===== 셀 20: 테스트 - Tool 필요 없는 일반 질문 =====
run_agent("안녕하세요!")


👤 사용자: 안녕하세요!

📡 Claude 응답 (stop_reason: end_turn)

🤖 Claude 응답 (Tool 미사용):
   안녕하세요! 😊

무엇을 도와드릴까요? 

현재 날씨, 시간 확인이나 계산이 필요하시면 언제든 말씀해 주세요!


'안녕하세요! 😊\n\n무엇을 도와드릴까요? \n\n현재 날씨, 시간 확인이나 계산이 필요하시면 언제든 말씀해 주세요!'

In [28]:
# ===== 셀 21: 테스트 - 모호한 질문 =====
run_agent("육경 나누기 일조?")


👤 사용자: 육경 나누기 일조?

📡 Claude 응답 (stop_reason: tool_use)
ToolUseBlock(id='toolu_01WkbHLL4KCkKhU8bgJiKdqz', input={'a': 6, 'b': 1, 'operation': 'divide'}, name='calculator', type='tool_use')

🔧 Tool 호출 요청:
   도구: calculator
   입력: {'a': 6, 'b': 1, 'operation': 'divide'}

⚡ Tool 실행 결과:
   6 ÷ 1 = 6.0

🤖 Claude 최종 응답:
   6(육) 나누기 1(일)은 **6**입니다.


'6(육) 나누기 1(일)은 **6**입니다.'

---

## 9. 정리

이번 단계에서 배운 내용:

| 개념 | 설명 |
|------|------|
| Tool | AI가 호출할 수 있는 외부 함수 |
| `name` | 도구 이름 (snake_case) |
| `description` | 도구 설명 (AI가 읽음) |
| `input_schema` | 입력 파라미터 정의 |
| `properties` | 각 파라미터의 타입과 설명 |
| `required` | 필수 파라미터 목록 |
| `enum` | 선택지 제한 |
| `stop_reason="tool_use"` | AI가 도구 호출을 원함 |

### 다음 단계

다음 노트북에서는 **Tool 실행**을 배웁니다.
- 도구를 실제로 실행하는 함수 작성
- 결과를 AI에게 전달하는 방법
- Agentic Loop 완성